## Generating SPARQL using Code LlaMa

### Setup of Code LLaMa 34B Quantized

In [1]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import time


model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        skip_special_tokens= True, 
        use_triton=use_triton,        
        quantize_config=None)

skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [2]:
# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",    
    do_sample=True,
    top_k=10,
    temperature=0.1,
    max_length=700,
    repetition_penalty=1.1,  
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,)

In [11]:
instruction = """[INST]
Task: convert question to SPARQL query for DBpedia knowledge graph.\n
Description: given an input question and a list of DBpedia URIs for the mentioned entities in the question. Write a correct SPARQL code to query these DBPedia URIs in DBpedia Knowledge graph. Please wrap your SPARQL code answer using ``` :
 \n

You can formulate your SPARQL query as the following examples. \n
<examples>
[question] : "what is the highest mountain in saxony germany?"
[entiteis] : ('saxony: <http://dbpedia.org/resource/Saxony>', 'germany: <http://dbpedia.org/resource/Germany>')

\n
SPARQL:
sparql: ```PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?m WHERE { ?m dbo:mountainRange dbr:Ore_Mountains ; dbo:elevation ?height } ORDER BY DESC(?height) LIMIT 1```"
</examples>

[/INST]
"""
    
def generate_sparql(question,entities): 
    
    sparql_prompt= instruction+'\n [question]:'+ question+' [entities] : ['+ ','.join(entities)+']'
    
    #response=pipe(sparql_prompt)
    response = pipe(sparql_prompt)[0]['generated_text']
    #print(response)
    #sparql= response[0]["generated_text"].split("<</SYS>>")[1]
    
    return response.split('[/INST]')[1].split('```')[1]

In [12]:
generate_sparql("what is the capital of Germany?","Germany")

'SELECT?capital WHERE { <http://dbpedia.org/resource/Germany> <http://dbpedia.org/ontology/capital>?capital }'